<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/Multistep_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install cohere -q

Instalacja biblioteki cohere za pomocą narzędzia pip. Flaga -q sprawia, że proces instalacji będzie przebiegał w trybie "quiet" - czyli z ograniczoną ilością komunikatów w konsoli.

Ten pakiet umożliwia korzystanie z modeli językowych Cohere, które służą m.in. do:
- generowania tekstu
- klasyfikacji tekstu
- wyszukiwania semantycznego
- embeddings (przekształcania tekstu na wektory liczbowe)


In [ ]:
import cohere
import json
from google.colab import userdata
import os

Importowanie niezbędnych bibliotek i modułów:

`cohere` - główna biblioteka do pracy z modelami Cohere

`json` - moduł do obsługi formatu JSON, który jest często używany przy wymianie danych

`kaggle_secrets` - moduł zawierający klasę UserSecretsClient, która pozwala na bezpieczne przechowywanie i dostęp do poufnych danych (jak klucze API) w środowisku Kaggle

`os` - moduł dostarczający funkcje do interakcji z systemem operacyjnym, używany często do operacji na ścieżkach plików i zmiennych środowiskowych

In [ ]:

api_key = userdata.get('cohereprod')
# os.environ['COHERE_API_KEY'] = api_key
co = cohere.Client(api_key)

Ten fragment kodu wykonuje następujące działania:

1. Pobiera klucz API z obiektu `userdata` używając metody `get()` i przypisuje go do zmiennej `api_key`. Klucz jest identyfikowany przez nazwę 'cohereprod'.

2. Zapisuje pobrany klucz API do zmiennej środowiskowej o nazwie 'COHERE_API_KEY'. Robi to poprzez przypisanie wartości do słownika `os.environ`, który przechowuje zmienne środowiskowe systemu.

3. Tworzy nową instancję klienta Cohere poprzez wywołanie `cohere.Client()` i przekazanie klucza API jako argumentu. Utworzony klient zostaje zapisany w zmiennej `co`.

Jest to standardowy sposób konfiguracji dostępu do API Cohere - biblioteki służącej do przetwarzania języka naturalnego. Kod przygotowuje wszystkie niezbędne elementy do wykonywania później operacji takich jak generowanie tekstu czy analiza semantyczna.

In [ ]:
class CFG:
    model = "command-r-plus"

# Functions

In [ ]:
def list_calendar_events(date: str):
  events = [{"start": "8:00", "end": "8:59"}, {"start": "9:00", "end": "9:59"}, {"start": "11:00", "end": "11:59"},{"start": "12:00", "end": "12:59"}]

  return {
        "existing_events": events
    }

Ta funkcja implementuje prosty mechanizm zarządzania wydarzeniami w kalendarzu:

`list_calendar_events(date: str)` - funkcja przyjmuje jeden parametr `date` typu string, który reprezentuje datę.

W środku funkcji zdefiniowana jest lista `events` zawierająca cztery wydarzenia w postaci słowników. Każde wydarzenie ma:
- `start` - czas rozpoczęcia wydarzenia (godzina:minuta)
- `end` - czas zakończenia wydarzenia (godzina:minuta)

Wydarzenia są zaplanowane na:
- 8:00 - 8:59
- 9:00 - 9:59
- 11:00 - 11:59
- 12:00 - 12:59

Funkcja zwraca słownik z jednym kluczem `existing_events`, którego wartością jest lista wydarzeń.

W tej implementacji data nie jest wykorzystywana - funkcja zawsze zwraca te same wydarzenia, niezależnie od podanej daty.

In [ ]:
def create_calendar_event(date: str, time: str, duration: int):

  return {
        "is_success": True,
        "message": f"Created a {duration} hour long event at {time} on {date}"
    }

Ta funkcja symuluje tworzenie nowego wydarzenia w kalendarzu:

`create_calendar_event(date: str, time: str, duration: int)` przyjmuje trzy parametry:
- `date` - data wydarzenia jako tekst
- `time` - godzina rozpoczęcia jako tekst
- `duration` - długość wydarzenia w godzinach jako liczba całkowita

Funkcja zwraca słownik zawierający dwa klucze:
- `is_success` - ustawiony na `True`, co oznacza że operacja się udała
- `message` - tekst potwierdzający utworzenie wydarzenia, zawierający wszystkie podane parametry

**Jest to uproszczona implementacja** - funkcja nie sprawdza poprawności danych ani nie weryfikuje konfliktów z istniejącymi wydarzeniami, zawsze zwraca sukces.

In [ ]:
functions_map = {
    "list_calendar_events": list_calendar_events,
    "create_calendar_event": create_calendar_event
}

Ta linia tworzy mapowanie (słownik) między nazwami funkcji a samymi funkcjami:

`functions_map` to słownik, gdzie:
- klucze to nazwy funkcji jako teksty (`"list_calendar_events"` i `"create_calendar_event"`)
- wartości to odpowiadające im funkcje (wcześniej zdefiniowane `list_calendar_events` i `create_calendar_event`)

Takie mapowanie jest często używane, gdy chcemy:
- dynamicznie wywoływać funkcje na podstawie ich nazw
- przekazywać informacje o dostępnych funkcjach do innych części systemu
- organizować wywołania funkcji w sposób elastyczny

W tym przypadku słownik pozwala na łatwe przekazanie informacji do modelu Cohere o tym, jakie funkcje są dostępne do użycia.

In [ ]:
tools = [
    {
      "name": "list_calendar_events",
      "description": "Returns a list of existing events for the specified date, including the start time and end time for each event.",
      "parameter_definitions": {
        "date": {
          "description": "the date to list events for, formatted as dd/mm/yy",
          "type": "str",
          "required": True
        }
      }
    },
    {
      "name": "create_calendar_event",
      "description": "Creates a new calendar event of the specified duration at the specified time and date. A new event cannot be created on the same time as an existing event.",
      "parameter_definitions": {
        "date": {
          "description": "the date on which the event starts, formatted as dd/mm/yy",
          "type": "str",
          "required": True
        },
        "time": {
          "description": "the time of the event, formatted using 24h military time formatting",
          "type": "str",
          "required": True
        },
        "duration": {
          "description": "the number of hours the event lasts for",
          "type": "float",
          "required": True
        }
      }
    }
]

Ta zmienna `tools` to lista zawierająca opisy dwóch narzędzi (funkcji) dostępnych w systemie kalendarza:

1. `list_calendar_events`:
- nazwa narzędzia do listowania wydarzeń
- opis: zwraca listę istniejących wydarzeń na podaną datę, wraz z czasem rozpoczęcia i zakończenia
- parametry: wymaga tylko daty w formacie dd/mm/rr

2. `create_calendar_event`:
- nazwa narzędzia do tworzenia wydarzeń
- opis: tworzy nowe wydarzenie o określonej długości w podanym czasie i dacie, z zastrzeżeniem że nie może kolidować z istniejącymi wydarzeniami
- parametry:
  - data rozpoczęcia (dd/mm/rr)
  - czas (format 24-godzinny)
  - długość wydarzenia w godzinach (liczba zmiennoprzecinkowa)

Ta struktura służy jako dokumentacja API i może być używana do:
- walidacji parametrów
- generowania dokumentacji
- automatycznego tworzenia interfejsów
- komunikacji z modelem językowym o dostępnych funkcjach

In [ ]:
def run_assistant(message, chat_history=None):
    if chat_history is None:
        chat_history = []

    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("-"*50)

    # Step 2: Generate tool calls (if any)
    response = co.chat(
        message=message,
        model = CFG.model,
        preamble=preamble,
        tools=tools,
        chat_history=chat_history
    )

    while response.tool_calls:
        tool_calls = response.tool_calls

        if response.text:
            print("Tool plan:")
            print(response.text,"\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("-"*50)

        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            tool_call = {"name": tc.name, "parameters": tc.parameters}
            tool_output = functions_map[tc.name](**tc.parameters)
            tool_results.append({"call": tool_call, "outputs": [tool_output]})

        # Step 4: Generate response and citations
        response = co.chat(
            message="",
            model = CFG.model,
            preamble=preamble,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history
        )

        # Append the current chat turn to the chat history
        chat_history = response.chat_history

    # Print final response
    print("Final response:")
    print(response.text)
    print("-"*50)

    # Print citations (if any)
    if response.citations:
        print("Citations:")
        for citation in response.citations:
            print(citation)
        print("\nCited Documents:")
        for document in response.documents:
            print(document)
        print("-"*50)

    return chat_history

Funkcja `run_assistant` zarządza konwersacją z asystentem Cohere. Działa w następujących krokach:

1. Inicjalizacja:
- tworzy pustą listę historii czatu jeśli nie została podana
- wyświetla pytanie użytkownika

2. Generowanie wywołań narzędzi:
- wysyła wiadomość do modelu Cohere z konfiguracją, narzędziami i historią czatu
- jeśli model proponuje użycie narzędzi, wyświetla plan i szczegóły wywołań

3. Wykonywanie narzędzi:
- dla każdego wywołania narzędzia:
  - przekazuje parametry do odpowiedniej funkcji z `functions_map`
  - zbiera wyniki działania

4. Generowanie odpowiedzi:
- wysyła kolejne zapytanie do modelu z wynikami narzędzi
- aktualizuje historię czatu

5. Wyświetlanie rezultatów:
- pokazuje końcową odpowiedź
- jeśli są cytowania, wyświetla je wraz z dokumentami źródłowymi

6. Zwraca zaktualizowaną historię czatu

Funkcja działa w pętli dopóki model sugeruje użycie kolejnych narzędzi, co pozwala na wieloetapowe rozwiązywanie zadań.

# Setup

In [ ]:
preamble="""## Task & Context
You are a calendar assistant who helps people schedule events on their calendar. You must make sure that a new event does not overlap with any existing event.
Today is Monday, 18th November, 2024
"""

# No multi-step

In [ ]:
chat_history = run_assistant("How many meetings do I have today")

Question:
How many meetings do I have today
--------------------------------------------------
Tool plan:
I will list the events for today and count how many there are. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '18/11/2024'}
--------------------------------------------------
Final response:
You have four meetings today.
--------------------------------------------------
Citations:
start=9 end=22 text='four meetings' document_ids=['list_calendar_events:0:2:0']

Cited Documents:
{'existing_events': '[{"end":"8:59","start":"8:00"},{"end":"9:59","start":"9:00"},{"end":"11:59","start":"11:00"},{"end":"12:59","start":"12:00"}]', 'id': 'list_calendar_events:0:2:0', 'tool_name': 'list_calendar_events'}
--------------------------------------------------


# Multi-step

In [ ]:
chat_history = run_assistant("Create an hour-long appointment for the first available free slot after 9am")

Question:
Create an hour-long appointment for the first available free slot after 9am
--------------------------------------------------
Tool plan:
I will first check the user's calendar for 18 November 2024 to see when they are free after 9am. Then, I will create an hour-long appointment for the first available slot. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '18/11/2024'}
--------------------------------------------------
Tool plan:
The user is free after 10am. I will now create an hour-long appointment for them at 10am. 

Tool calls:
Tool name: create_calendar_event | Parameters: {'date': '18/11/2024', 'duration': 1, 'time': '10:00'}
--------------------------------------------------
Final response:
I've created an hour-long appointment for you at 10am.
--------------------------------------------------
Citations:
start=16 end=37 text='hour-long appointment' document_ids=['create_calendar_event:0:4:0']
start=49 end=53 text='10am' document_ids=['create_calen

In [ ]:
for turn in chat_history:
    print(turn,"\n")

role='USER' message='Create an hour-long appointment for the first available free slot after 9am' tool_calls=None 

role='CHATBOT' message="I will first check the user's calendar for 18 November 2024 to see when they are free after 9am. Then, I will create an hour-long appointment for the first available slot." tool_calls=[ToolCall(name='list_calendar_events', parameters={'date': '18/11/2024'})] 

role='TOOL' tool_results=[ToolResult(call=ToolCall(name='list_calendar_events', parameters={'date': '18/11/2024'}), outputs=[{'existing_events': [{'end': '8:59', 'start': '8:00'}, {'end': '9:59', 'start': '9:00'}, {'end': '11:59', 'start': '11:00'}, {'end': '12:59', 'start': '12:00'}]}])] 

role='CHATBOT' message='The user is free after 10am. I will now create an hour-long appointment for them at 10am.' tool_calls=[ToolCall(name='create_calendar_event', parameters={'date': '18/11/2024', 'duration': 1, 'time': '10:00'})] 

role='TOOL' tool_results=[ToolResult(call=ToolCall(name='create_calenda

# Multi-step parallel

In [ ]:
chat_history = run_assistant("Create two hour-long appointments for any available time between 8am to 6pm")

Question:
Create two hour-long appointments for any available time between 8am to 6pm
--------------------------------------------------
Tool plan:
I will list the events for today, then create two one-hour appointments at times when there are no events. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '18/11/2024'}
--------------------------------------------------
Tool plan:
There are four events today: one from 8:00-8:59, one from 9:00-9:59, one from 11:00-11:59, and one from 12:00-12:59. I will create two one-hour appointments at times when there are no events. 

Tool calls:
Tool name: create_calendar_event | Parameters: {'date': '18/11/2024', 'duration': 1, 'time': '10:00'}
Tool name: create_calendar_event | Parameters: {'date': '18/11/2024', 'duration': 1, 'time': '13:00'}
--------------------------------------------------


# State management - memory

In [ ]:
chat_history = run_assistant("Considering the new appointments you made, when is my next available time?", chat_history)

In [ ]:
for turn in chat_history:
    print(turn,"\n")